In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/My\ Drive/Colab\ Notebooks/apex-codes/entity_sum

/content/drive/My Drive/Colab Notebooks/apex-codes/entity_sum


In [ ]:
# works for BART and T5 only
#!pip3 install transformers==2.8.0

# For BART, T5, Pegasus
!pip3 install transformers==3.5.0

# For Prophetnet
#!pip3 install git+https://github.com/huggingface/transformers

!pip3 install torch==1.7.0
!pip3 install sentencepiece

     |████████████████████████████████| 1.3 MB 4.2 MB/s 
     |████████████████████████████████| 2.9 MB 44.2 MB/s 
     |████████████████████████████████| 1.1 MB 53.8 MB/s 
     |████████████████████████████████| 895 kB 36.7 MB/s 
     |████████████████████████████████| 776.7 MB 4.3 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.


In [ ]:
# Checking out the GPU we have access to
!nvidia-smi

Sat Jan  8 02:42:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Importing stock libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

#from transformers import T5Tokenizer, T5ForConditionalGeneration
#from transformers import BartTokenizer, BartForConditionalGeneration
#from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from transformers import ProphetNetTokenizer, ProphetNetForConditionalGeneration

In [ ]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
device

'cuda'

### Read pubmed dataset

In [ ]:
import json

DATA_PATH = "pubmed_dataset/pubmed-dataset"
data = []
with open(f"{DATA_PATH}/val.txt") as f:
  for line in f:
    data.append(json.loads(line))

In [ ]:
data[0]['abstract_text']

['<S> background and aim : there is lack of substantial indian data on venous thromboembolism ( vte ) . </S>',
 '<S> the aim of this study was to provide real - world information on patient characteristics , management strategies , clinical outcomes , and temporal trends in vte.subjects and methods : multicentre retrospective registry involving 549 medical records of patients with confirmed diagnosis of vte ( deep vein thrombosis [ dvt ] confirmed by doppler ultrasonography ; pulmonary embolism [ pe ] by computed tomography , pulmonary angiography and/or v / q scan ) from 2006 to 2010 at three indian tertiary care hospitals.results:acute dvt without pe , acute dvt with pe , and pe alone were reported in 64% ( 352/549 ) , 23% ( 124/549 ) , and 13% ( 73/549 ) patients , respectively . </S>',
 '<S> mean age was 47 ( 16 ) years , and 70% were males . </S>',
 '<S> h / o dvt ( 34% ) , surgery including orthopedic surgery ( 28% ) , trauma ( 16% ) , and immobilization > 3 days ( 14% ) were the

In [ ]:
"".join(data[0]['abstract_text']).replace('<S>', '').replace('</S>', '')

' background and aim : there is lack of substantial indian data on venous thromboembolism ( vte ) .  the aim of this study was to provide real - world information on patient characteristics , management strategies , clinical outcomes , and temporal trends in vte.subjects and methods : multicentre retrospective registry involving 549 medical records of patients with confirmed diagnosis of vte ( deep vein thrombosis [ dvt ] confirmed by doppler ultrasonography ; pulmonary embolism [ pe ] by computed tomography , pulmonary angiography and/or v / q scan ) from 2006 to 2010 at three indian tertiary care hospitals.results:acute dvt without pe , acute dvt with pe , and pe alone were reported in 64% ( 352/549 ) , 23% ( 124/549 ) , and 13% ( 73/549 ) patients , respectively .  mean age was 47 ( 16 ) years , and 70% were males .  h / o dvt ( 34% ) , surgery including orthopedic surgery ( 28% ) , trauma ( 16% ) , and immobilization > 3 days ( 14% ) were the most common risk factors for vte .  hyp

In [ ]:
"".join(data[0]['article_text'])

"approximately , one - third of patients with symptomatic vte manifests pe , whereas two - thirds manifest dvt alone .both dvt and pe can be clinically silent ( asymptomatic ) and hence not suspected .if undiagnosed , asymptomatic vte can lead to chronic venous disease or recurrent vte and long - term debilitating sequelae such as postthrombotic syndrome and chronic thromboembolic pulmonary hypertension .vte is not only disabling but also prolongs hospital stay and increases the cost of treatment .along with myocardial infarction and arrhythmia ( due to electrolyte imbalance ) , pe is one of the commonest causes of sudden unexplained deaths in hospitalized patients .it is estimated that 20 million cases of lower extremity dvt occur in the usa alone .the prevailing notion that the incidence of vte in asians is less than that in the western population has been disproved by recent studies .the incidence of postoperative dvt in indian patients undergoing major lower limb surgery is as high

### Parse the article abstract and body (text)

In [ ]:
final_data = []

for article in data:
  article_abstract = "".join(article['abstract_text']).replace('<S>', '').replace('</S>', '')
  article_text = "".join(article['article_text'])

  final_data.append([article_text, article_abstract])

df = pd.DataFrame(final_data, columns=['article_text', 'article_abstract'])

In [ ]:
df.head()

,article_text,article_abstract
0,"approximately , one - third of patients with s...",background and aim : there is lack of substan...
1,there is an epidemic of stroke in low and midd...,backgroundthe questionnaire for verifying str...
2,\n cardiovascular diseases account for the hig...,\n background : timely access to cardiovascul...
3,results of a liquid culturing system ( bd bact...,to determine differences in the ability of my...
4,the need for magnetic resonance imaging ( mri ...,aimsour aim was to evaluate the potential for...


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df['article_text'][0]

"approximately , one - third of patients with symptomatic vte manifests pe , whereas two - thirds manifest dvt alone .both dvt and pe can be clinically silent ( asymptomatic ) and hence not suspected .if undiagnosed , asymptomatic vte can lead to chronic venous disease or recurrent vte and long - term debilitating sequelae such as postthrombotic syndrome and chronic thromboembolic pulmonary hypertension .vte is not only disabling but also prolongs hospital stay and increases the cost of treatment .along with myocardial infarction and arrhythmia ( due to electrolyte imbalance ) , pe is one of the commonest causes of sudden unexplained deaths in hospitalized patients .it is estimated that 20 million cases of lower extremity dvt occur in the usa alone .the prevailing notion that the incidence of vte in asians is less than that in the western population has been disproved by recent studies .the incidence of postoperative dvt in indian patients undergoing major lower limb surgery is as high

In [ ]:
len(word_tokenize(df['article_abstract'][0]))

355

In [ ]:
len(df['article_text'].values)

6633

In [ ]:
sum_text = 0
sum_abstract = 0

for idx, row in df.iterrows():
  sum_text += len(df['article_text'][idx])
  sum_abstract += len(df['article_abstract'][idx])

avg_text = sum_text / float(6633)
avg_abstract = sum_abstract /float(6633)

In [ ]:
print(avg_text)
print(avg_abstract)

17746.687471732246
1235.1302578018997


In [ ]:
df.article_text = 'summarize: ' + df.article_text

In [ ]:
df.head()

,article_text,article_abstract
0,"summarize: approximately , one - third of pati...",background and aim : there is lack of substan...
1,summarize: there is an epidemic of stroke in l...,backgroundthe questionnaire for verifying str...
2,summarize: \n cardiovascular diseases account ...,\n background : timely access to cardiovascul...
3,summarize: results of a liquid culturing syste...,to determine differences in the ability of my...
4,summarize: the need for magnetic resonance ima...,aimsour aim was to evaluate the potential for...


### Select which model to fine-tune

In [ ]:
# Sections of config

# Defining some key variables that will be used later on in the training

#tokenizer = T5Tokenizer.from_pretrained("t5-base")
#tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
#tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')
tokenizer = ProphetNetTokenizer.from_pretrained('microsoft/prophetnet-large-uncased')

MAX_LEN = 512
SUMMARY_LEN = 150
TRAIN_BATCH_SIZE = 2
VALID_BATCH_SIZE = 2
EPOCHS = 2
LEARNING_RATE = 1e-4

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/141 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:341: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [ ]:
# Creating a custom dataset for reading the dataframe and loading it into the dataloader to pass it to the neural network at a later stage 
# for finetuning the model and to prepare it for predictions

class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.article_abstract = self.data.article_abstract
        self.article_text = self.data.article_text

    def __len__(self):
        return len(self.article_abstract)

    def __getitem__(self, index):
        article_text = str(self.article_text[index])
        article_text = ' '.join(article_text.split())

        article_abstract = str(self.article_abstract[index])
        article_abstract = ' '.join(article_abstract.split())

        source = self.tokenizer.batch_encode_plus([article_text], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([article_abstract], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')


        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [ ]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state=42).reset_index(drop=True)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)

FULL Dataset: (6633, 2)
TRAIN Dataset: (5306, 2)
TEST Dataset: (1327, 2)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
#model = T5ForConditionalGeneration.from_pretrained("t5-base")
#model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
#model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')
model = ProphetNetForConditionalGeneration.from_pretrained('microsoft/prophetnet-large-uncased', )

model = model.to(device)

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:341: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def train(epoch):
  model.train()
  for _, data in enumerate(training_loader, 0):
    y = data['target_ids'].to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)

    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)

    loss = outputs[0]
    
    if _%500==0:
        print(f'Epoch: {epoch}, Loss:  {loss.item()}')
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
def _get_config_file(model_path, model_name):
    # Name of the file for storing hyperparameter details
    return os.path.join(model_path, model_name + ".config")

def _get_model_file(model_path, model_name):
    # Name of the file for storing network parameters
    return os.path.join(model_path, model_name + ".tar")

def load_model(model_path, model_name, net=None):
    """
    Loads a saved model from disk.

    Inputs:
        model_path - Path of the checkpoint directory
        model_name - Name of the model (str)
        net - (Optional) If given, the state dict is loaded into this model. Otherwise, a new model is created.
    """
    config_file, model_file = _get_config_file(model_path, model_name), _get_model_file(model_path, model_name)
    assert os.path.isfile(config_file), f"Could not find the config file \"{config_file}\". Are you sure this is the correct path and you have your model config stored here?"
    assert os.path.isfile(model_file), f"Could not find the model file \"{model_file}\". Are you sure this is the correct path and you have your model stored here?"
    with open(config_file, "r") as f:
        config_dict = json.load(f)
    if net is None:
        act_fn_name = config_dict["act_fn"].pop("name").lower()
        act_fn = act_fn_by_name[act_fn_name](**config_dict.pop("act_fn"))
        net = BaseNetwork(act_fn=act_fn, **config_dict)
    net.load_state_dict(torch.load(model_file, map_location=device))
    return net

def save_model(model, model_path, model_name):
    """
    Given a model, we save the state_dict and hyperparameters.

    Inputs:
        model - Network object to save parameters from
        model_path - Path of the checkpoint directory
        model_name - Name of the model (str)
    """
    config_dict = model.config
    os.makedirs(model_path, exist_ok=True)
    config_file, model_file = _get_config_file(model_path, model_name), _get_model_file(model_path, model_name)
    with open(config_file, "w") as f:
        json.dump(config_dict, f)
    torch.save(model.state_dict(), model_file)

### Call to the train method above

In [ ]:
for epoch in range(EPOCHS):
  train(epoch)

#########################
# save the trained pytorch model----change the named of the model in accordance with the baseline model currently being worked on
model_name = "pubmed-Prophetnet"    # changes with each baseline model or the proposed approach's name
model_path = f"pubmed-pytorch_models"
os.makedirs(model_path, exist_ok=True)

# Save entire model
# Specify a path
model_path = f"pubmed-pytorch_models/{model_name}.pt"
torch.save(model, model_path)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch: 0, Loss:  4.889352798461914
Epoch: 0, Loss:  4.417749881744385
Epoch: 0, Loss:  4.777266025543213
Epoch: 0, Loss:  5.323818683624268
Epoch: 0, Loss:  4.716580867767334
Epoch: 0, Loss:  4.053532600402832
Epoch: 1, Loss:  4.166281700134277
Epoch: 1, Loss:  3.9656007289886475
Epoch: 1, Loss:  4.206107139587402
Epoch: 1, Loss:  4.481459617614746
Epoch: 1, Loss:  4.732577323913574
Epoch: 1, Loss:  3.566321611404419


## Saving the model as a checkpoint

In [ ]:
#model = T5ForConditionalGeneration.from_pretrained("t5-base")
#model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
#model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')

In [ ]:
!pip3 install torch==1.7.0
!pip3 install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-3z0_743t
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-3z0_743t
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached tokenizers-0.10.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Created wheel for transformers: filename=transformers-4.13.0.dev0-py3-none-any.whl size=3101540 sha256=6eb76432068b2d04117e815d98103868d04fc2b0f40f3a8ed600aa97f8534f5d
  Stored in directory: /tmp/pip-ephem-wheel-cache-qyjjme4q/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.5.2
    Uninstalling tokenizers-0.5.2:
      Successfully uninstalled tokenizers-0.5.2
  Attempting uninstall: transformers
    Found exis

In [ ]:
import torch
import transformers
import os

In [ ]:
modelName = "Pegasus"
model_name = f"pubmed-{modelName}"

# model path
model_path = f"../entity_sum/pubmed-pytorch_models/{model_name}.pt"

model = torch.load(model_path)

os.makedirs(f'{modelName}-checkpoints')
model.save_pretrained(f'../entity_sum/{modelName}-checkpoints/')

### Saving/Loading entire model to the file

In [ ]:
# save the trained pytorch model----change the named of the model in accordance with the baseline model currently being worked on
from torchvision import models
from torchsummary import summary


model_name = "pubmed-T5"
model_path = f"pubmed-pytorch_models"

# model path
#model_path = f"pubmed-pytorch_models/{model_name}.pth"
model_path = f"pubmed-pytorch_models/{model_name}.pt"


#model = model.load_state_dict(torch.load(model_path))
model = torch.load(model_path)

In [ ]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [ ]:
model.state_dict()

OrderedDict([('shared.weight',
              tensor([[ -0.7576,   0.6001,  -2.4332,  ...,   1.2522,  -0.7822,   3.5235],
                      [ 11.3739,  -4.8733,   9.0615,  ...,   4.8424,  14.3902,  -5.7711],
                      [-16.6135,  11.0992, -20.8572,  ...,  10.6604,  22.2536,  24.9938],
                      ...,
                      [  2.2344,   6.7500, -11.0625,  ..., -11.3125,  13.5625,  16.6250],
                      [  4.2500,   5.1250, -12.2500,  ..., -11.9375,  13.5000,  17.0000],
                      [  4.0625,   6.9688, -12.2500,  ..., -11.3750,  11.9375,  16.6250]],
                     device='cuda:0')),
             ('encoder.embed_tokens.weight',
              tensor([[ -0.7576,   0.6001,  -2.4332,  ...,   1.2522,  -0.7822,   3.5235],
                      [ 11.3739,  -4.8733,   9.0615,  ...,   4.8424,  14.3902,  -5.7711],
                      [-16.6135,  11.0992, -20.8572,  ...,  10.6604,  22.2536,  24.9938],
                      ...,
                   

### Experiment with generating summaries

In [ ]:
# First, read a text from the file

t5_prepared_Text = "summarize: " + preprocess_text
tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)

# summmarize 
summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=70,
                                    max_length=120,
                                    early_stopping=True)

final_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
print(final_summary)

In [ ]:
'''
# save the trained pytorch model
model_name = "pubmed-BART"
model_path = f"pytorch_models"
os.makedirs(model_path, exist_ok=True)

# Specify a path
model_path = f"pytorch_models/{model_name}/{model_name}.pt"

# Save
#torch.save(net, PATH)
torch.save(model, model_path)
'''

In [ ]:
def writer(predictions, actuals):
    # output_path = r'/content/drive/My Drive/Colab Notebooks/'
    # pred_file = output_path+'preds.txt'
    # actual_file = output_path+'actuals.txt'
    # with open(pred_file, 'w') as f:
    #     for pred in predictions:
    #         f.write('%s\n' % pred)

    # with open(actual_file, 'w') as f:
    #     for actual in actuals:
    #         f.write('%s\n' % actual)
    my_dict = {
        'Actual Headline': actuals,
        'Prediction': predictions       
        }
    final_output = pd.DataFrame(my_dict)

    return final_output


In [ ]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration

# tokenizer = T5Tokenizer.from_pretrained('t5-small')
# model = T5ForConditionalGeneration.from_pretrained('t5-small')
# input_ids = tokenizer.encode("Hello, my dog is cute", return_tensors="pt")  # Batch size 1
# outputs = model(input_ids=input_ids, decoder_input_ids=input_ids, lm_labels=input_ids)
# loss, prediction_scores = outputs[:2]

# tokenizer = T5Tokenizer.from_pretrained('t5-small')
# model = T5ForConditionalGeneration.from_pretrained('t5-small')
# input_ids = tokenizer.encode_plus("summarize: Hello, my dog is cute", return_tensors="pt")  # Batch size 1
# outputs = model.generate(input_ids['input_ids'])
# pred = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in outputs]

In [ ]:
# print(f'The loss is: {loss}')
# print(f'The Prediction score is {prediction_scores}')
# print(f'The output is {outputs}')
# print(f'The final output is {pred}')